In [34]:
import mne
import matplotlib.pyplot as plt 
from scipy.signal import welch
from scipy.stats import f_oneway
import numpy as np 
import pandas as pd 
from utils import quikcap_32, fix_chans
from mne.preprocessing import ICA, corrmap, create_ecg_epochs, create_eog_epochs
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import MultiComparison
from tqdm import tqdm

### Only for Curry AAS 

In [27]:
# Define conditions
conds = ['OUT', 'EPI', 'BFFE']

# Define subjects 
subjects = ['S1', 'S2']

# Define algorithms 
algos = ['RAW', 'AAS']

# Define frequency bands for EEG
delta_lo, delta_hi = 1, 4
theta_lo, theta_hi = 4, 8
alpha_lo, alpha_hi = 8, 12
beta_lo, beta_hi = 12, 35

# Create stats dictionary
stats = {cond: {subject: {algo: {} for algo in algos} for subject in subjects} for cond in conds}

for cond in conds: # Iterating over conditions
    for algo in algos: # Iterating over algorithms
        for subj in subjects: # Iterating subjects 
            pass
            ## Read raw EEG ##
            try:
                eeg = mne.io.read_raw_curry(f'EEG-MRI/real/{subj}_{algo}_{cond}.cdt', preload=True)
            except:
                print(f"Error: {subj}_{algo}_{cond} not in filepath. Moving on...")
                continue
            print(f"\nAdding {subj}_{algo}_{cond} to Dictionary")
            
            # fix_chans function
            
            # 1. Marks EKG and EMG as bad (not used)
            # 2. Assigns VEOG, HEOG to EOG channel
            # 3. Drops EKG and EMG (required for quikcap_32 montage to be applied)
            fix_chans(eeg)
            
            # Crop 
            # No automatic cropping yet 
            if cond == 'OUT':
                if subj == 'S1':
                    eeg.crop(tmin=142,tmax=262)
                else:
                    eeg.crop(tmin=748, tmax=838)
            # Automatic cropping 
            elif (cond == 'BFFE' or cond == 'EPI'):
                events, event_id = mne.events_from_annotations(eeg) # Get trigger events 
                if subj == 'S2' and cond == 'EPI':
                    # to account for spurious events
                    start_sample = events[3][0] # First event; Start time (unit samples)
                else:
                    start_sample = events[0][0] # First event; Start time (unit samples)
                med_diff = np.median(np.diff(events[:, 0])).astype(int) # Get TR (in unit samples)
                print('med_diff (TR): ', med_diff)
                end_sample = events[-1][0] + med_diff # Last event + TR; End time (unit samples)
                print('\nstart_sample', start_sample)
                print('end_sample', end_sample)
                srate = eeg.info['sfreq']
                start_time = start_sample / srate # convert to s
                print('start_time', start_time) 
                end_time = end_sample / srate # convert to s 
                print('end_time', end_time)

                eeg.crop(tmin=start_time, tmax=end_time)

            # store raw EEG in dictionary
            stats[cond][subj][algo]['raw'] = eeg
            
            ## Filtering + Pre-processing ## 
            
            # bandpass filtering from 0-40 Hz
            print('\nBandpass Filtering from 0 - 40 Hz')
            eeg_bp = eeg.copy().filter(l_freq=0, h_freq=40)
            
            # ICA EOG removal
            print('\nStarting ICA...')
            # Filtering to remove slow drifts:
            slowfilt_eeg = eeg_bp.copy().filter(l_freq=1, h_freq=None)

            # ICA created with 15 components
            ica = ICA(n_components=15, max_iter="auto")

            # Fitting ICA
            ica.fit(slowfilt_eeg)

            # Finding which ICs match with EOG 
            ica.exclude = []
            eog_indices, eog_scores = ica.find_bads_eog(slowfilt_eeg)
            ica.exclude = eog_indices

            # Applying ICA 
            ica.apply(eeg_bp)
            print('\nICA Finished.')
            
            # Store processed EEG
            stats[cond][subj][algo]['raw_processed'] = eeg_bp
            
            ## PSD Calculations ## 
            
            data = eeg_bp.get_data(picks=["O1", "O2", "Oz"]) # (n_channels, n_times). Picks to isolate alpha oscillations
            
            # Computer PSD 
            srate = eeg.info['sfreq']
            psd, freqs = mne.time_frequency.psd_array_welch(data, sfreq=srate, n_fft=int(srate))
            
            # Compute normalised PSD
            psd_norm = psd / psd.sum() # sum() over n_times 
            
            # Avg PSD across channels
            psd_avg = np.mean(psd, axis=0)
            psd_norm_avg = np.mean(psd_norm, axis=0)
            
            # Store PSD and Freqs
            stats[cond][subj][algo]['freqs'] = freqs
            stats[cond][subj][algo]['psd_abs'] = psd_avg
            stats[cond][subj][algo]['psd_norm'] = psd_norm_avg
            
            # Indices for freq bands
            i_delta_lo = np.argmin((delta_lo - freqs)**2)
            i_delta_hi = np.argmin((delta_hi - freqs)**2)
            i_theta_lo = np.argmin((theta_lo - freqs)**2)
            i_theta_hi = np.argmin((theta_hi - freqs)**2)
            i_alpha_lo = np.argmin((alpha_lo - freqs)**2)
            i_alpha_hi = np.argmin((alpha_hi - freqs)**2)
            i_beta_lo = np.argmin((beta_lo - freqs)**2)
            i_beta_hi = np.argmin((beta_hi - freqs)**2)
            
            # absolute and normalised power dictionarys 
            powers_abs = {
                'delta': psd_avg[i_delta_lo:i_delta_hi],
                'theta': psd_avg[i_theta_lo:i_theta_hi],
                'alpha': psd_avg[i_alpha_lo:i_alpha_hi],
                'beta': psd_avg[i_beta_lo:i_beta_hi]
            }

            # mean absolute freq. band power
            stats[cond][subj][algo]['mean_delta'] = np.mean(powers_abs['delta'])
            stats[cond][subj][algo]['mean_theta'] = np.mean(powers_abs['theta'])
            stats[cond][subj][algo]['mean_alpha'] = np.mean(powers_abs['alpha'])
            stats[cond][subj][algo]['mean_beta'] = np.mean(powers_abs['beta'])
            
            # relative freq. band power 
            stats[cond][subj][algo]['rel_delta'] = powers_abs['delta'].sum() / psd_avg.sum()
            stats[cond][subj][algo]['rel_theta'] = powers_abs['theta'].sum() / psd_avg.sum()
            stats[cond][subj][algo]['rel_alpha'] = powers_abs['alpha'].sum() / psd_avg.sum()
            stats[cond][subj][algo]['rel_beta'] = powers_abs['beta'].sum() / psd_avg.sum()
            
            
            # 1.5s epochs for 30 s duration
            stats[cond][subj][algo]['epochs_mean_alpha'] = []
            stats[cond][subj][algo]['epochs_rel_alpha'] = []
            
            srate = int(srate)
            
            # Avg over picks 
            eeg_avg = np.mean(data, axis=0)
            
            # crop to 30 s 
            eeg_30 =  eeg_avg[:30*srate] 
            
            # get epoch start times
            starts = np.arange(0, (30*srate), (1.5*srate)).astype(int)
            
            # append mean and rel alpha for 1.5 s epochs over 30 s
            for start in starts:
                start_end = start + int(1.5*srate)
                epoch = eeg_30[start:start_end]
                freqs, psd = welch(epoch, fs=srate, nperseg=srate)
                
                i_alpha_lo = np.argmin((alpha_lo - freqs)**2)
                i_alpha_hi = np.argmin((alpha_hi - freqs)**2)
                
                mean_abs_power = np.mean(psd[i_alpha_lo:i_alpha_hi])
                stats[cond][subj][algo]['epochs_mean_alpha'].append(mean_abs_power)
                
                alpha_rel_power = psd[i_alpha_lo:i_alpha_hi].sum() / psd.sum()
                stats[cond][subj][algo]['epochs_rel_alpha'].append(alpha_rel_power)

Leaving device<->head transform as None (no landmarks found)
Reading 0 ... 2930999  =      0.000 ...   293.100 secs...
Event file not found. No Annotations set.

Adding S1_RAW_OUT to Dictionary

Bandpass Filtering from 0 - 40 Hz
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (0.330 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s



Starting ICA...
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 33001 samples (3.300 s)



[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Fitting ICA to data using 32 channels (please be patient, this may take a while)


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s


... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 100000 samples (10.000 s)



[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Applying ICA to Raw instance
    Transforming to ICA space (15 components)
    Zeroing out 0 ICA components
    Projecting back using 32 PCA components

ICA Finished.
Effective window size : 1.000 (s)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


### PSD visualisation code

In [40]:
def pltspectra(freqs, *psds, labels, title):
    """
    Function to automatically create overlapping PSD plots using MatPlotLib
    
    input vars:
    freqs = freqs from scipy.signal.welch()
    *psds = psd from scipy.signal.welch(). Ensure freqs. match the PSDs you use.
    labels = labels (list of strings) 
    title = plot title (string)"""
    colourcycle = ['k', 'b', 'r', 'g', 'c', 'm', 'y']
    plt.figure(figsize=(10,5))
    
    for i, psd in enumerate(psds):
        plt.semilogy(freqs[i_delta_lo:i_beta_hi], psd[i_delta_lo:i_beta_hi], 
                        colourcycle[i], label=labels[i])
    
    plt.xlabel("freq. (Hz)", fontsize=14)
    plt.ylabel("power " + r"$\mu V^2/Hz$", fontsize=14)
    plt.legend(loc='upper right', fontsize=16)
    plt.title(title, fontsize=16)
    plt.tight_layout()
    plt.show()

### Band powers summary table

#### Absolute Mean Band Power

In [29]:
subjects = ['S1', 'S2']
conditions = ['OUT', 'EPI', 'BFFE']
algos = ['RAW', 'AAS']

rows = ['Delta (1-4 Hz)',
        'Theta (4-8 Hz)',
        'Alpha (8-12 Hz)',
        'Beta (12-30 Hz)']

powers = ['mean_delta',
          'mean_theta',
          'mean_alpha',
          'mean_beta']

columns = pd.MultiIndex.from_product([subjects, conditions, algos], names=['Subject', 'Condition', 'Avg Abs. Band Power'])
df = pd.DataFrame(index=rows, columns=columns)

for subj in subjects:
    for cond in conditions:
        for algo in algos:
            subject_data = []
            for metric_idx, metric in enumerate(rows):
                power = powers[metric_idx]
                try:
                    value = stats[cond][subj][algo][power]
                except:
                    value = np.nan
                subject_data.append(value)
            df.loc[:, (subj, cond, algo)] = subject_data

# Drop columns w/ NaNs
df = df.dropna(axis=1, how='all')

df = df.applymap(lambda x: "{:.3e}".format(x) if not pd.isna(x) else "")

df


Subject                     S1                                              \
Condition                  OUT        EPI                  BFFE              
Avg Abs. Band Power        RAW        RAW        AAS        RAW        AAS   
Delta (1-4 Hz)       7.047e-12  6.203e-11  2.714e-09  1.336e-10  6.160e-11   
Theta (4-8 Hz)       2.109e-12  1.164e-10  2.658e-10  1.244e-10  8.021e-11   
Alpha (8-12 Hz)      3.892e-12  1.233e-10  1.188e-10  1.063e-10  5.504e-11   
Beta (12-30 Hz)      8.135e-13  3.846e-11  2.238e-11  1.066e-11  5.671e-12   

Subject                     S2                                              
Condition                  OUT        EPI                  BFFE             
Avg Abs. Band Power        RAW        RAW        AAS        RAW        AAS  
Delta (1-4 Hz)       1.954e-11  5.437e-11  1.099e-10  4.933e-10  9.185e-11  
Theta (4-8 Hz)       6.627e-12  3.349e-11  4.035e-11  1.760e-10  4.778e-11  
Alpha (8-12 Hz)      4.556e-11  2.101e-11  1.992e-11  5.388e-11  1.848e-11  
Beta (12-30 Hz)      4.193e-13  1.717e-10  3.536e-12  4.301e-11  3.731e-12

#### Relative Band Power

In [32]:
subjects = ['S1', 'S2']
conditions = ['OUT', 'EPI', 'BFFE']
algos = ['RAW', 'AAS']

rows = ['Delta (1-4 Hz)',
        'Theta (4-8 Hz)',
        'Alpha (8-12 Hz)',
        'Beta (12-30 Hz)']

powers = ['rel_delta',
          'rel_theta',
          'rel_alpha',
          'rel_beta']

columns = pd.MultiIndex.from_product([subjects, conditions, algos], names=['Subject', 'Condition', 'Relative Band Power'])
df = pd.DataFrame(index=rows, columns=columns)

for subj in subjects:
    for cond in conditions:
        for algo in algos:
            subject_data = []
            for metric_idx, metric in enumerate(rows):
                power = powers[metric_idx]
                try:
                    value = stats[cond][subj][algo][power]
                except:
                    value = np.nan
                subject_data.append(value)
            df.loc[:, (subj, cond, algo)] = subject_data

# Drop columns w/ NaNs
df = df.dropna(axis=1, how='all')

df = df.applymap(lambda x: "{:.2f}".format(x) if not pd.isna(x) else "")

df


Subject                S1                            S2                    \
Condition             OUT   EPI        BFFE         OUT   EPI        BFFE   
Relative Band Power   RAW   RAW   AAS   RAW   AAS   RAW   RAW   AAS   RAW   
Delta (1-4 Hz)       0.32  0.07  0.78  0.25  0.21  0.21  0.02  0.47  0.38   
Theta (4-8 Hz)       0.13  0.17  0.10  0.31  0.37  0.09  0.02  0.23  0.18   
Alpha (8-12 Hz)      0.24  0.18  0.05  0.26  0.25  0.65  0.01  0.11  0.06   
Beta (12-30 Hz)      0.28  0.31  0.05  0.15  0.15  0.03  0.58  0.12  0.25   

Subject                    
Condition                  
Relative Band Power   AAS  
Delta (1-4 Hz)       0.42  
Theta (4-8 Hz)       0.29  
Alpha (8-12 Hz)      0.11  
Beta (12-30 Hz)      0.13

## Stats code 

### 1st One-Way ANOVA
- Outside v raw fMRI v raw fCI
- From 1.5 s epochs from 30 s

In [38]:
# Absolute mean alpha power 
# Hyp -> Sig

x0 = stats['OUT']['S2']['RAW']['epochs_mean_alpha']
x1 = stats['EPI']['S2']['RAW']['epochs_mean_alpha']
x2 = stats['BFFE']['S2']['RAW']['epochs_mean_alpha']

print('One-way ANOVA for abs. mean alpha power across Out, raw fMRI, raw fCI')

F, p = f_oneway(x0, x1, x2)
print('F-statistic: ', F)
print('p value: ', p )
alpha = 0.05 # significance level
if p < alpha:
    print(f"\tSignificant group differences found, p={p:.3f}")
else:
    print(f"\tNo significant group differences found, p={p:.3f}")
    
    
# Relative alpha power 
# Hyp -> Sig

x3 = stats['OUT']['S2']['RAW']['epochs_rel_alpha']
x4 = stats['EPI']['S2']['RAW']['epochs_rel_alpha']
x5 = stats['BFFE']['S2']['RAW']['epochs_rel_alpha']

print("\nOne-way ANOVA for relative alpha power across Out, raw fMRI, raw fCI")

F, p = f_oneway(x3, x4, x5)
print('F-statistic: ', F)
print('p value: ', p )
alpha = 0.05 # significance level
if p < alpha:
    print(f"\tSignificant group differences found, p={p:.3f}")
else:
    print(f"\tNo significant group differences found, p={p:.3f}")
    

One-way ANOVA for abs. mean alpha power across Out, raw fMRI, raw fCI
F-statistic:  12.24558127465293
p value:  3.7651627433942905e-05
	Significant group differences found, p=0.000

One-way ANOVA for relative alpha power across Out, raw fMRI, raw fCI
F-statistic:  633.1264921227489
p value:  1.1901761599834653e-39
	Significant group differences found, p=0.000


### 2nd One-Way ANOVA
- Outside v AAS fMRI v AAS fCI
- From 1.5 s epochs from 30 s

In [39]:
# Absolute mean alpha power 
# Hyp -> Non-sig

x0 = stats['OUT']['S2']['RAW']['epochs_mean_alpha']
x1 = stats['EPI']['S2']['AAS']['epochs_mean_alpha']
x2 = stats['BFFE']['S2']['AAS']['epochs_mean_alpha']

print('One-way ANOVA for abs. mean alpha power across Out, AAS fMRI, AAS fCI')

F, p = f_oneway(x0, x1, x2)
print('F-statistic: ', F)
print('p value: ', p )
alpha = 0.05 # significance level
if p < alpha:
    print(f"\tSignificant group differences found, p={p:.3f}")
else:
    print(f"\tNo significant group differences found, p={p:.3f}")
    
    
# Relative alpha power 
# Hyp -> Non-sig

x3 = stats['OUT']['S2']['RAW']['epochs_rel_alpha']
x4 = stats['EPI']['S2']['AAS']['epochs_rel_alpha']
x5 = stats['BFFE']['S2']['AAS']['epochs_rel_alpha']

print("\nOne-way ANOVA for relative alpha power across Out, AAS fMRI, AAS fCI")

F, p = f_oneway(x0, x1, x2)
print('F-statistic: ', F)
print('p value: ', p )
alpha = 0.05 # significance level
if p < alpha:
    print(f"\tSignificant group differences found, p={p:.3f}")
else:
    print(f"\tNo significant group differences found, p={p:.3f}")
    

One-way ANOVA for abs. mean alpha power across Out, AAS fMRI, AAS fCI
F-statistic:  21.56389569564763
p value:  1.0632276010315279e-07
	Significant group differences found, p=0.000

One-way ANOVA for relative alpha power across Out, AAS fMRI, AAS fCI
F-statistic:  21.56389569564763
p value:  1.0632276010315279e-07
	Significant group differences found, p=0.000
